In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession
.builder
.appName("SparkSQLExampleApp")
.getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-C8F58JN:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1618297744768)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3d55a19a


In [2]:
val df = spark.read.format("csv")
.option("inferSchema", "true")
.option("header", "true")
.load("C:/Users/GAME/Desktop/departuredelays.csv")

df: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [3]:
df.createOrReplaceTempView("us_delay_flights_tbl")

In [5]:
spark.sql("""SELECT distance, origin, destination
FROM us_delay_flights_tbl WHERE distance > 1000
ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [6]:
spark.sql("""SELECT date, delay, origin, destination
FROM us_delay_flights_tbl
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'
ORDER by delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [7]:
spark.sql("""SELECT delay, origin, destination,
CASE
WHEN delay > 360 THEN 'Very Long Delays'
WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
WHEN delay = 0 THEN 'No Delays'
ELSE 'Early'
END AS Flight_Delays
FROM us_delay_flights_tbl
ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



In [8]:
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

res4: org.apache.spark.sql.DataFrame = []


In [16]:
spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT,
distance INT, origin STRING, destination STRING)")

<console>: 2: error: unclosed string literal

In [26]:
val schema="date STRING, delay INT, distance INT, origin STRING, destination STRING"
val csv_file = "C:/Users/GAME/Desktop/departuredelays.csv"
val flights_df = spark.read.csv(csv_file)
flights_df.write.saveAsTable("managed_us_delay_flights_tbl")

schema: String = date STRING, delay INT, distance INT, origin STRING, destination STRING
csv_file: String = C:/Users/GAME/Desktop/departuredelays.csv
flights_df: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 3 more fields]


In [27]:
spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT,
distance INT, origin STRING, destination STRING)
USING csv OPTIONS (PATH
'C:/Users/GAME/Desktop/departuredelays.csv')""")

res11: org.apache.spark.sql.DataFrame = []


In [41]:
val df_sfo = spark.sql("""SELECT date, delay, origin, destination FROM
us_delay_flights_tbl WHERE origin = "SFO"""")


df_sfo: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 2 more fields]


In [42]:
val df_jfk = spark.sql("""SELECT date, delay, origin, destination FROM
us_delay_flights_tbl WHERE origin = "JFK"""")

df_jfk: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 2 more fields]


In [43]:
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")

In [45]:
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view")

res15: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 2 more fields]


In [46]:
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

res16: Boolean = true


In [47]:
spark.catalog.listDatabases()
spark.catalog.listTables()
spark.catalog.listColumns("us_delay_flights_tbl")

res17: org.apache.spark.sql.Dataset[org.apache.spark.sql.catalog.Column] = [name: string, description: string ... 4 more fields]


In [48]:
val usFlightsDF = spark.sql("SELECT * FROM us_delay_flights_tbl")
val usFlightsDF2 = spark.table("us_delay_flights_tbl")

usFlightsDF: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]
usFlightsDF2: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [51]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

In [58]:
val file = """C:/Users/GAME/Desktop/parquet/2010-summary.parquet/"""
val df = spark.read.format("parquet").load(file)

file: String = C:/Users/GAME/Desktop/parquet/2010-summary.parquet/
df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [59]:
df.createOrReplaceTempView("us_delay_flights_tbl")

In [60]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [61]:
df.write.format("parquet")
.mode("overwrite")
.save("/tmp/data/parquet/df_parquet")

<console>: 31: error: value asd is not a member of Unit

In [16]:
import org.apache.spark:spark-avro_2.12:3.1.1
val file = "C:/Users/GAME/Desktop/avro/*"
val df = spark.read.format("avro").load(file)
df.createOrReplaceTempView("us_delay_flights_tbl")
spark.sql("SELECT * FROM us_delay_flights_tbl").show()
df.write.format("avro")
.mode("overwrite")
.save("C:/Users/GAME/Desktop/prueba")

<console>: 2: error: ';' expected but ':' found.

In [63]:
import org.apache.spark.sql.avro.functions._

<console>: 25: error: object avro is not a member of package org.apache.spark.sql

In [6]:
val ciudad = df.select("date", "delay", "origin", "destination").where("delay > 120 and origin = 'SFO' and destination = 'ORD'")

ciudad: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: int, delay: int ... 2 more fields]


In [7]:
ciudad.show(20, false)

+-------+-----+------+-----------+
|date   |delay|origin|destination|
+-------+-----+------+-----------+
|1011410|124  |SFO   |ORD        |
|1022330|326  |SFO   |ORD        |
|1021410|190  |SFO   |ORD        |
|1101410|184  |SFO   |ORD        |
|1190925|297  |SFO   |ORD        |
|1241110|139  |SFO   |ORD        |
|1301800|167  |SFO   |ORD        |
|1011237|122  |SFO   |ORD        |
|1032258|163  |SFO   |ORD        |
|1031920|193  |SFO   |ORD        |
|1031755|396  |SFO   |ORD        |
|1071040|279  |SFO   |ORD        |
|1161210|225  |SFO   |ORD        |
|1221040|215  |SFO   |ORD        |
|1261104|258  |SFO   |ORD        |
|1020720|145  |SFO   |ORD        |
|1021205|154  |SFO   |ORD        |
|1031550|131  |SFO   |ORD        |
|1041205|126  |SFO   |ORD        |
|1051205|320  |SFO   |ORD        |
+-------+-----+------+-----------+
only showing top 20 rows



In [13]:
import org.apache.spark.sql.functions.{when, _}
import spark.sqlContext.implicits._
val ciudad = df.select("delay", "origin", "destination")
.when(col("delay") > 360, "Very Long Delays")
.when(col("delay") > 120 && col (delay) < 360, "Long Delays")
.when(col("delay") > 60 && col (delay) < 120, "Short Delays")
.when(col("delay") > 0 && col (delay) < 60, "Tolerable Delays")
.when(col("delay") === 0, "No Delays")
.otherwise("Early")
.orderBy(desc("origin", "delay"))

<console>: 32: error: value when is not a member of org.apache.spark.sql.DataFrame

In [14]:
val nueva_fecha = (new df.withColumn("fecha", to_timestamp(col("date"), "MM/dd/yyyy")). drop("date"))
nueva_fecha.select("DAY(fecha)", "MONTH(fecha)", "SUM(delay)").groupBy("DAY(fecha)", "MONTH(fecha)")
spark.sql("SELECT DAY(fecha), MONTH(fecha), SUM(delay) from us_delay_flights_tbl group by DAY(fecha), MONTH(fecha)")

<console>: 27: error: type withColumn is not a member of org.apache.spark.sql.DataFrame

In [17]:
val file = "C:/Users/GAME/Desktop/json/*"
val df = spark.read.format("csv").load(file)

file: String = C:/Users/GAME/Desktop/json/*
df: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 1 more field]


In [18]:
val file = "C:/Users/GAME/Desktop/json/*"
val df = spark.read.format("csv").load(file)
df.createOrReplaceTempView("us_delay_flights_tbl")
spark.sql("SELECT * FROM us_delay_flights_tbl").show()
df.write.format("json")
.mode("overwrite")
.save("C:/Users/GAME/Desktop/prueba")

+--------------------+--------------------+------------+
|                 _c0|                 _c1|         _c2|
+--------------------+--------------------+------------+
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...| "count":15}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...|  "count":1}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...|"count":344}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...| "count":15}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...| "count":62}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...|  "count":1}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...| "count":62}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...|"count":588}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...| "count":40}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...|  "count":1}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...|"count":325}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...| "count":39}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...| "count":64}|
|{"ORIGIN_COUNTRY_...|"DEST_COUNTRY_NAM...|  "count":1}|
|{"ORIGIN_COUNTRY_...|"DEST_COU

file: String = C:/Users/GAME/Desktop/json/*
df: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 1 more field]


In [19]:
val file = "C:/Users/GAME/Desktop/csv/*"
val df = spark.read.format("csv").load(file)
df.createOrReplaceTempView("us_delay_flights_tbl")
spark.sql("SELECT * FROM us_delay_flights_tbl").show()
df.write.format("csv")
.mode("overwrite")
.save("C:/Users/GAME/Desktop/prueba")

+--------------------+-------------------+-----+
|                 _c0|                _c1|  _c2|
+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United State

file: String = C:/Users/GAME/Desktop/csv/*
df: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 1 more field]


In [ ]:
val file = "C:/Users/GAME/Desktop/orc/*"
val df = spark.read.format("orc").load(file)
df.createOrReplaceTempView("us_delay_flights_tbl")
spark.sql("SELECT * FROM us_delay_flights_tbl").show()
df.write.format("orc")
.mode("overwrite")
.save("C:/Users/GAME/Desktop/prueba")